In [1]:
%pip install pillow

     |████████████████████████████████| 2.1MB 758kB/s eta 0:00:01    |▎                               | 20kB 129kB/s eta 0:00:16     |██████████████████████████      | 1.7MB 758kB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
%conda install tensorflow keras

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - keras
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |              mkl           2 KB  defaults
    absl-py-0.7.1              |           py37_0         154 KB  conda-forge
    astor-0.7.1                |             py_0          22 KB  conda-forge
    binutils_impl_linux-64-2.31.1|       h6176602_1         3.9 MB  defaults
    binutils_linux-64-2.31.1   |       h6176602_8         148 KB  defaults
    c-ares-1.15.0              |    h516909a_1001         100 KB  conda-forge
    gast-0.2.2                 |             py_0          10 KB  conda-forge
    gcc

In [2]:
import sys
print(sys.path)
python_path = ':'.join(sys.path)

['/home/jovyan', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '', '/opt/conda/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages/IPython/extensions', '/home/jovyan/.ipython']


In [4]:
from konduitserving.inference import *
import random
from konduitserving.json_utils import json_with_type
import json

input_names = ['default']
output_names = ['default']
port = 65535
parallel_inference_config = ParallelInferenceConfig(workers=1)
serving_config = ServingConfig(http_port=port,
                                   log_timings=True,
                                   parallel_inference_config=parallel_inference_config)

python_config = PythonConfig(
        python_path=python_path,
        python_code_path='"/usr/share/exec.py',
        python_inputs={'img_path': 'STRING'},
        python_outputs={'label': 'STRING','proba':'FLOAT'}
)

python_pipeline_step = PythonPipelineStep(input_names=input_names,
                                              output_names=output_names,  
                                           input_schemas=({'default': ['String']}),
                                           output_schemas=({'default': ['String','Float']}),
                                           input_column_names={'default': ['img_path']},
                                             output_column_names={'default': ['label','proba']},
                                              python_configs={'default': python_config})

inference = InferenceConfiguration(serving_config=serving_config,
                                       pipeline_steps=[python_pipeline_step])


json_config = json_with_type(inference)
print(json.dumps(json_config, indent=4, sort_keys=True))


{
    "@type": "InferenceConfiguration",
    "pipelineSteps": [
        {
            "@type": "PythonPipelineStep",
            "inputColumnNames": {
                "default": [
                    "img_path"
                ]
            },
            "inputNames": [
                "default"
            ],
            "inputSchemas": {
                "default": [
                    "String"
                ]
            },
            "outputColumnNames": {
                "default": [
                    "label",
                    "proba"
                ]
            },
            "outputNames": [
                "default"
            ],
            "outputSchemas": {
                "default": [
                    "String",
                    "Float"
                ]
            },
            "pythonConfigs": {
                "default": {
                    "@type": "PythonConfig",
                    "pythonCodePath": "\"/usr/share/exec.py",
                    "pyt

In [5]:
!wget https://joinaikido.com/wp-content/uploads/2013/07/African_Bush_Elephant_1-783x1024.jpg

--2019-08-20 06:17:15--  https://joinaikido.com/wp-content/uploads/2013/07/African_Bush_Elephant_1-783x1024.jpg
Resolving joinaikido.com (joinaikido.com)... 35.197.111.104
Connecting to joinaikido.com (joinaikido.com)|35.197.111.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191472 (187K) [image/jpeg]
Saving to: ‘African_Bush_Elephant_1-783x1024.jpg’

African_Bush_Elepha 100%[===================>] 186.98K   261KB/s    in 0.7s    

2019-08-20 06:17:16 (261 KB/s) - ‘African_Bush_Elephant_1-783x1024.jpg’ saved [191472/191472]



In [6]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

img_path = 'African_Bush_Elephant_1-783x1024.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
decoded = decode_predictions(preds, top=3)[0]
id,label,proba = decoded[0]

Using TensorFlow backend.
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

40960/35363 [==================================] - 0s 4us/step
